# Scraping Boxofficemojo

### Scraping studio links

In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup as bs
import time

response = requests.get('https://www.boxofficemojo.com/studio/')
html = response.text
soup = bs(html,'lxml')

table_node = soup.find_all('table', attrs = {'border': '0', 'cellspacing': '1', 'cellpadding': '5'})
link_nodes = soup.find_all('a', href = True)
studios = []
for link in link_nodes:
    if '/studio/chart/' in link['href']:
        studios.append(link['href'].split('&studio=')[1].split('.htm')[0])

### Function for scraping a studio's movies

In [2]:
def get_movies(studio):
    i = 1
    while True:
        tables = pd.read_html('https://www.boxofficemojo.com/studio/chart/?page=' + str(i) + '&yr=&view=company&view2=allmovies&studio=' + studio + '.htm&p=.htm', header = 0)
        if len(tables) == 2: break
        dfs = tables[1]
        dfs.drop(dfs.tail(4).index, inplace = True)
        dfs.columns = ['boRank', 'title', 'studioAcronym', 'boWorldwide', 'totalTheaters', 'boOpening', 'openingTheaters', 'openDate'] 
        if i == 1:
            dfn = dfs.copy()
        else:
            dfn = dfn.append(dfs) 
        i += 1
        time.sleep(1)
    try:
        return dfn
    except:
        return None

### Scraping all movies

In [3]:
i = 1
for studio in studios:
    if i == 1:
        dff = get_movies(studio)
    else:
        try:
            dff = dff.append(get_movies(studio))
        except:
            pass
    print(i)
    i += 1
    time.sleep(1)
dff

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129


,boRank,title,studioAcronym,boWorldwide,totalTheaters,boOpening,openingTheaters,openDate
0,1,Star Wars: The Force Awakens,BV,"$936,662,225",4134,"$247,966,675",4134,12/18/15
1,2,Avengers: Endgame,BV,"$858,177,594",4662,"$357,115,007",4662,4/26/19
2,3,Black Panther,BV,"$700,059,566",4084,"$202,003,951",4020,2/16/18
3,4,Avengers: Infinity War,BV,"$678,815,482",4474,"$257,698,183",4474,4/27/18
4,5,Marvel's The Avengers,BV,"$623,357,910",4349,"$207,438,708",4349,5/4/12
5,6,Star Wars: The Last Jedi,BV,"$620,181,382",4232,"$220,009,584",4232,12/15/17
6,7,Incredibles 2,BV,"$608,581,744",4410,"$182,687,905",4410,6/15/18
7,8,Rogue One: A Star Wars Story,BV,"$532,177,324",4157,"$155,081,681",4157,12/16/16
8,9,Beauty and the Beast (2017),BV,"$504,014,165",4210,"$174,750,616",4210,3/17/17
9,10,The Lion King (2019),BV,"$501,335,434",4802,"$191,770,759",4725,7/19/19


### Structuring data

In [33]:
dff = pd.read_pickle('bom_new_raw.pkl')

In [34]:
def convert_year(string):
    try:
        string = string.split('/')[2]
        if string[0] in '23456789':
            return int('19' + string)
        else:
            return int('20' + string)
    except:
        return None
    
def numerize_money(string):
    if isinstance(string, str):
        try:
            return int(float(string.replace('$', '').replace(',', '')))
        except:
            return None
    else:
        return None


dff['boOpening'] = dff['boOpening'].apply(numerize_money)
dff['boWorldwide'] = dff['boWorldwide'].apply(numerize_money)
dff['year'] = dff['openDate'].apply(convert_year)
dff['year'] = dff['openDate'].apply(convert_year)
dff = dff.drop('openDate', axis = 1)

dff.to_pickle('bom_clean.pkl')

In [35]:
dff

,boRank,title,studioAcronym,boWorldwide,totalTheaters,boOpening,openingTheaters,year
0,1,Star Wars: The Force Awakens,BV,936662225,4134,247966675.0,4134,2015.0
1,2,Avengers: Endgame,BV,858177594,4662,357115007.0,4662,2019.0
2,3,Black Panther,BV,700059566,4084,202003951.0,4020,2018.0
3,4,Avengers: Infinity War,BV,678815482,4474,257698183.0,4474,2018.0
4,5,Marvel's The Avengers,BV,623357910,4349,207438708.0,4349,2012.0
5,6,Star Wars: The Last Jedi,BV,620181382,4232,220009584.0,4232,2017.0
6,7,Incredibles 2,BV,608581744,4410,182687905.0,4410,2018.0
7,8,Rogue One: A Star Wars Story,BV,532177324,4157,155081681.0,4157,2016.0
8,9,Beauty and the Beast (2017),BV,504014165,4210,174750616.0,4210,2017.0
9,10,The Lion King (2019),BV,501335434,4802,191770759.0,4725,2019.0
